In [1]:
import pandas as pd
import requests
from datetime import datetime

In [4]:
def extract(URL, area, inicio, fim, endpoint="cargaverificada"):    
    data_inicio = datetime.strptime(inicio, '%Y-%m-%d')
    data_fim = datetime.strptime(fim, '%Y-%m-%d')
    
    dataframes = [] 
    
    inicio_atual = data_inicio
    while inicio_atual <= data_fim:
        fim_atual = min(inicio_atual + pd.Timedelta(days=90), data_fim)

        params = {
            'cod_areacarga': area,
            'dat_inicio': inicio_atual.strftime('%Y-%m-%d'),
            'dat_fim': fim_atual.strftime('%Y-%m-%d')
        }

        url = f"{URL}/{endpoint}"
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            dados = response.json()
            if dados: 
                df = pd.DataFrame(dados)
                dataframes.append(df)
            else:
                print(f"Aviso: Dados não encontrados para o período {params['dat_inicio']} a {params['dat_fim']}.")
        else:
            print(f"Erro na requisição para o período {params['dat_inicio']} a {params['dat_fim']}: {response.status_code} - {response.text}")

        inicio_atual = fim_atual + pd.Timedelta(days=1)

    if dataframes:
        final_df = pd.concat(dataframes, ignore_index=True)
        final_df.drop_duplicates(inplace=True) 
        return final_df
    else:
        return pd.DataFrame()

In [63]:
def transform(df, programada=False):
    coluna_carga = 'val_cargaglobal'
    if programada:
        coluna_carga = 'val_cargaglobalprogramada'

    df_transformed = df.copy()
    df_transformed['din_referenciautc'] = pd.to_datetime(df_transformed['din_referenciautc']).dt.tz_localize(None) - pd.Timedelta(hours=3)

    df_transformed.rename(columns={'din_referenciautc': 'datetime'}, inplace=True)

    df_transformed = df_transformed[[
        'datetime', 'cod_areacarga', coluna_carga,
    ]]

    return df_transformed

In [ ]:
areas = {'SECO': None, 'S': None, 'NE': None, 'N': None}

for key in areas:
    areas[key] = extract(URL="https://apicarga.ons.org.br/prd/", area=key, inicio="2024-01-01", fim="2025-08-14")
    areas[key] = transform(areas[key])

SIN = pd.concat([areas['SECO'], areas['S'], areas['NE'], areas['N']])
SIN = SIN.pivot(index='datetime', columns='cod_areacarga', values='val_cargaglobal')

SIN['SIN'] = SIN.sum(axis=1)
SIN = SIN[SIN['SIN'] != 0].copy()

In [72]:
SIN

cod_areacarga,N,NE,S,SECO,SIN
datetime,,,,,
2024-01-01 00:30:00,6543.3926,11438.925,9565.011,34999.273,62546.6016
2024-01-01 01:00:00,6551.2544,11553.903,9686.818,35000.560,62792.5354
2024-01-01 01:30:00,6599.0960,11672.290,9703.138,34979.176,62953.7000
2024-01-01 02:00:00,6647.7812,11748.536,9629.959,34987.250,63013.5262
2024-01-01 02:30:00,6709.9300,11834.165,9490.616,34801.312,62836.0230
...,...,...,...,...,...
2025-08-14 09:30:00,8186.6196,12718.270,14369.248,41988.750,77262.8876
2025-08-14 10:00:00,8338.8270,12825.584,14413.127,42559.176,78136.7140
2025-08-14 10:30:00,8447.8180,12968.138,14586.116,43319.530,79321.6020


In [44]:
SIN

cod_areacarga,N,NE,S,SECO,SIN
datetime,,,,,
2024-01-01 00:30:00,6543.3926,11438.925,9565.011,34999.273,62546.6016
2024-01-01 01:00:00,6551.2544,11553.903,9686.818,35000.560,62792.5354
2024-01-01 01:30:00,6599.0960,11672.290,9703.138,34979.176,62953.7000
2024-01-01 02:00:00,6647.7812,11748.536,9629.959,34987.250,63013.5262
2024-01-01 02:30:00,6709.9300,11834.165,9490.616,34801.312,62836.0230
...,...,...,...,...,...
2025-08-14 09:30:00,8186.6196,12718.270,14369.248,41988.750,77262.8876
2025-08-14 10:00:00,8338.8270,12825.584,14413.127,42559.176,78136.7140
2025-08-14 10:30:00,8447.8180,12968.138,14586.116,43319.530,79321.6020


In [64]:
areas = {'SECO': None, 'S': None, 'NE': None, 'N': None}

for key in areas:
    areas[key] = extract(URL="https://apicarga.ons.org.br/prd/", area=key, inicio="2024-01-01", fim="2025-08-14", endpoint="cargaprogramada")
    areas[key] = transform(areas[key], programada=True)

programada = pd.concat([areas['SECO'], areas['S'], areas['NE'], areas['N']])
programada = programada.pivot(index='datetime', columns='cod_areacarga', values='val_cargaglobalprogramada')

programada['SIN'] = programada.sum(axis=1)

In [65]:
programada

cod_areacarga,N,NE,S,SECO,SIN
datetime,,,,,
2024-01-01 00:30:00,6857.4680,13331.854,10457.9450,36734.844,67382.1110
2024-01-01 01:00:00,6902.5513,13321.628,10450.7640,36748.707,67423.6503
2024-01-01 01:30:00,6951.9844,13158.060,10433.5480,36642.367,67185.9594
2024-01-01 02:00:00,7011.4270,13040.970,10345.2720,36535.164,66932.8330
2024-01-01 02:30:00,7067.8335,12951.420,10206.3955,36407.050,66632.6990
...,...,...,...,...,...
2025-08-14 22:00:00,8915.0630,14637.041,14957.8270,46342.203,84852.1340
2025-08-14 22:30:00,8891.8980,14542.840,14445.5480,44804.550,82684.8360
2025-08-14 23:00:00,8978.5980,14384.760,13853.9220,43429.895,80647.1750
